In [2]:
import sys
import os
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
# ADD OTHER FOLDERS TO THIS LIST TO ADD THEM TO THE sys.path
modules_to_add = [""]

this_file = os.path.abspath('')

for module in modules_to_add:
    p = Path(this_file).parent / module 
    if p.exists():
        sys.path.append(str(p))
        print(f"ADDED: {p}")
    else:
        print(f"ERROR: {p} doesn't exist")

ADDED: /home/gerardozinno/Desktop/Tesi/Code/mlem


In [4]:
print(sys.path)

['/home/gerardozinno/Desktop/Tesi/Code/mlem/notebooks', '/home/gerardozinno/.pyenv/versions/3.9.9/lib/python39.zip', '/home/gerardozinno/.pyenv/versions/3.9.9/lib/python3.9', '/home/gerardozinno/.pyenv/versions/3.9.9/lib/python3.9/lib-dynload', '', '/home/gerardozinno/.pyenv/versions/3.9.9/envs/ml-environment/lib/python3.9/site-packages', '/home/gerardozinno/.pyenv/versions/3.9.9/envs/ml-environment/lib/python3.9/site-packages/IPython/extensions', '/home/gerardozinno/.ipython', '/home/gerardozinno/Desktop/Tesi/Code/mlem']


## Loading and scaling the data

In [5]:
from sklearn.datasets import load_iris

In [6]:
data, target = load_iris(return_X_y=True, as_frame=True)

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

## Fitting a model

In [8]:
import torch
import numpy as np
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

In [9]:
X_train, X_test, y_train, y_test = train_test_split(data_scaled, target.to_numpy(), test_size=0.2, random_state=123)

In [10]:
class IrisScaledDataset(Dataset):
    def __init__(self, features, target):
        assert(len(features) == len(target))
        self.feat = features.astype(np.float32)
        self.targ = target
    def __len__(self):
        return len(self.feat)
    
    def __getitem__(self, idx):
        f = self.feat[idx]
        t = self.targ[idx]
        return f,t

In [11]:
train_data = IrisScaledDataset(X_train, y_train)
test_data = IrisScaledDataset(X_test, y_test)

train_dataloader = DataLoader(train_data, batch_size=16)
test_dataloader = DataLoader(test_data, batch_size=16)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [13]:
class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_dim, 50)
        self.layer2 = nn.Linear(50, 50)
        self.layer3 = nn.Linear(50, 3)
        
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        #x = F.softmax(self.layer3(x), dim=1)
        x = self.layer3(x)
        return x
    
model     = Model(X_train.shape[1])
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn   = nn.CrossEntropyLoss()
model

Model(
  (layer1): Linear(in_features=4, out_features=50, bias=True)
  (layer2): Linear(in_features=50, out_features=50, bias=True)
  (layer3): Linear(in_features=50, out_features=3, bias=True)
)

In [16]:
from blackboxes.pytorch.utilities import train

In [17]:
train(model, optimizer, loss_fn,train_dataloader, test_dataloader, epochs=20, device=device)

Epoch: 0, Training Loss: 1.05, Test Loss: 0.958195, accuracy = 0.60
Epoch: 1, Training Loss: 0.94, Test Loss: 0.835086, accuracy = 0.80
Epoch: 2, Training Loss: 0.85, Test Loss: 0.717046, accuracy = 0.80
Epoch: 3, Training Loss: 0.75, Test Loss: 0.605446, accuracy = 0.80
Epoch: 4, Training Loss: 0.67, Test Loss: 0.505629, accuracy = 0.83
Epoch: 5, Training Loss: 0.59, Test Loss: 0.424377, accuracy = 0.90
Epoch: 6, Training Loss: 0.52, Test Loss: 0.361878, accuracy = 0.90
Epoch: 7, Training Loss: 0.47, Test Loss: 0.316326, accuracy = 0.93
Epoch: 8, Training Loss: 0.42, Test Loss: 0.282618, accuracy = 0.97
Epoch: 9, Training Loss: 0.38, Test Loss: 0.255458, accuracy = 0.97
Epoch: 10, Training Loss: 0.35, Test Loss: 0.232464, accuracy = 1.00
Epoch: 11, Training Loss: 0.32, Test Loss: 0.212865, accuracy = 1.00
Epoch: 12, Training Loss: 0.29, Test Loss: 0.196167, accuracy = 1.00
Epoch: 13, Training Loss: 0.26, Test Loss: 0.181852, accuracy = 1.00
Epoch: 14, Training Loss: 0.24, Test Loss: 0

In [18]:
from mlem.black_box import PyTorchBlackBox

bb = PyTorchBlackBox(model)

In [25]:
n = 100
preds = bb.predict(X_test[:n]) 
preds == y_test[:n]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True,  True])

In [35]:
print(bb.predict(X_test))
print(y_test)

[1 2 2 1 0 2 1 0 0 1 2 0 1 2 2 2 0 0 1 0 0 1 0 2 0 0 0 2 2 0]
[1 2 2 1 0 2 1 0 0 1 2 0 1 2 2 2 0 0 1 0 0 2 0 2 0 0 0 2 2 0]


In [27]:
bb.predict_proba(X_test[:4])

array([[-4.2191644 ,  1.3768513 ,  1.1773652 ],
       [-5.9572577 ,  0.478063  ,  3.4632816 ],
       [-6.0839725 ,  0.15437214,  3.9273834 ],
       [-0.9967218 ,  1.2332355 , -1.1004293 ]], dtype=float32)

# Saving the model and its data

In [18]:
ROOT = Path("../pretrained/")
MODEL_NAME      = "iris_pytorch.tar"
SAVE_PATH = ROOT / MODEL_NAME

In [19]:
x_train, x_test, y_train, y_test = X_train, X_test, y_train, y_test
dic = {
    'model_state_dict' : model.state_dict(),
    'x_train' : x_train,
    'y_train' : y_train,
    'x_test': x_test,
    'y_test': y_test
}
torch.save(dic, SAVE_PATH)
print(f"SAVED: {SAVE_PATH}")

SAVED: ../pretrained/iris_pytorch.tar
